In [ ]:
import numpy as np
from PIL import Image
import zipfile
with zipfile.ZipFile('/content/Data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

images = np.ones([2400,785])

In [ ]:
def getXTilda():
    path = "/content/Data/Train/"
    X = np.ones([2400, 785 ])
    label = np.genfromtxt("/content/Training Labels.txt", delimiter = ",")
    for i in range(2400):
        curri = i+1
        newpath = path + str(curri) + ".jpg"
        img = Image.open(newpath)
        image = np.asarray(img).flatten()
        image = np.append(image,label[i])
    X[i] =  image
    return X
x = getXTilda()

In [ ]:
def getT(number):
    T = np.ones([2400])
    T *= -1
    start =  number*240
    for i in range (240):
        T[start+i] *=-1
    return T

getT(0)

In [ ]:
def getA(X):
    Xtranspose = X.transpose()
    dotProduct = Xtranspose.dot(X)
    inverse = np.linalg.pinv(dotProduct)
    A = inverse.dot(Xtranspose)
    return A

In [ ]:
def getW(A , T):
  return A.dot(T)

In [ ]:
def test(path):
    X = getXTilda()
    A = getA(X)
    resultLabels = np.ones([200])

    for i in range(200):
        curri = i+1
        newpath = path + str(curri) + ".jpg"
        img = Image.open(newpath)
        image = np.asarray(img).flatten()
        image = np.append(image, [1])
        outputWeights = np.zeros([10])
        for j in range(10):
            currW = getW(A, getT(j));
            outputWeights[j] = image.dot(currW)
        maxElement = np.amax(outputWeights)
        index = np.where(outputWeights == maxElement)
        resultLabels[i] = index[0][0]
    return resultLabels



In [ ]:
originalLabels = np.genfromtxt("/content/Test Labels.txt", delimiter = ",")
originalLabels.size

In [ ]:
def confusionMat(original, result):
    cm = np.zeros([10, 10])
    for i in range(len(original)):
        cm[int(original[i])][int(result[i])] += 1

    return cm.astype(int)

In [ ]:
resultLabels = test("/content/Data/Test/")
resultLabels.size

In [ ]:
cm = confusionMat(originalLabels, test("/content/Data/Test/"))
cm

In [ ]:
import matplotlib.pyplot as plt
def plot_cm(y_true, y_pred, classes, cm, title=None):
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax
np.set_printoptions(precision=2)
class_names = ["0","1","2","3","4","5","6","7","8","9"]
plot_cm(originalLabels, resultLabels, classes=class_names, cm = cm,
                      title='Normalized confusion matrix')
plt.savefig("cm.png", dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)
plt.show()